In [ ]:
import scanpy as sc
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [45]:
import scvi

In [46]:
scvi.settings.dl_num_workers = 47

In [4]:
adata=sc.read("../../process/pre-intergration/big_data/20241019_lop10_hvg.h5ad")

In [31]:
adata_count = sc.read("../../process/pre-intergration/big_data/20241019_mergeall_filter_gene_step1.h5ad")

In [36]:
adata_count[adata.obs_names,adata.var_names].X[1:10,1:10].toarray()

array([[0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 2., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [38]:
adata.layers["counts"] = adata_count[adata.obs_names,adata.var_names].X.copy()

In [39]:
adata.X = adata.layers["counts"]
adata.raw = adata
raw = adata.raw.to_adata()
raw.X = adata.layers['counts']
adata.raw = raw
adata.X[1:10,1:10].toarray()

array([[0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 2., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 2., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 2., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [5]:
sc.settings.figdir="../../results/integration/20241019_scANVI/"

In [9]:
batch = pd.read_csv("../../processed_data/attributeName/batch.csv",index_col=0)

In [13]:
adata.obs["batch"] = batch["x"].values

In [14]:
adata

AnnData object with n_obs × n_vars = 339489 × 36303
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'coarse_anno_1', 'total_counts', 'log10_total_counts', 'n_genes_detected', 'mito_frac', 'ribo_frac', 'compl', 'size_factors', 'Sample', 'Project', 'Core_datasets', 'Mandibular_Maxillary', 'Molar_Incisor', 'Tooth position', 'Histology', 'Sex', 'Gene Type', 'Knockout_gene', 'Cre', 'Treatment', 'FACs', 'Age', 'Stage', 'Development stage', 'Disease', 'Strain', 'Dissociation_enzyme', 'Machine', 'Species', 'Related assay', 'Journal', 'Data Source', 'Data location', 'new_totals_log', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'batch'
    var: 'gene_symbols', 'n_cells', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_

In [15]:
condition_key = 'batch'
cell_type_key = 'coarse_anno_1'
unlabeled_category = "unlabeled"


In [23]:
adata = adata[:,adata.var["highly_variable"]].copy()

In [30]:
adata.layers["counts"][1:10,1:10].toarray()

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 4.59, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.59, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 3.8 , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 2.33, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.29, 2.58, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 2.44, 0.  ]])

In [40]:
sca.models.SCVI.setup_anndata(adata, batch_key=condition_key, labels_key=cell_type_key)

/home/zhanglab/micromamba/envs/py311/lib/python3.11/site-packages/scvi/data/fields/_layer_field.py:115: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


In [41]:
vae = sca.models.SCVI(
    adata,
    n_layers=1,n_latent=30,dispersion="gene-batch",
    dropout_rate=0.1,gene_likelihood="zinb",
    encode_covariates=True,
    deeply_inject_covariates=False,
    use_layer_norm="both",
    use_batch_norm="none",
)

In [47]:
vae.train(max_epochs = 100)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Training:   0%|          | 0/100 [00:00<?, ?it/s]

INFO: 
Detected KeyboardInterrupt, attempting graceful shutdown ...
INFO:lightning.pytorch.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
scanvae = sca.models.SCANVI.from_scvi_model(vae, unlabeled_category ="Unkown")
scanvae.train(max_epochs=50)

In [ ]:
reference_latent = sc.AnnData(scanvae.get_latent_representation())
reference_latent.obs[cell_type_key] = adata.obs[cell_type_key].tolist()
reference_latent.obs["batch"] = adata.obs[condition_key].tolist()
reference_latent.obs["Project"] = adata.obs["Project"].tolist()

In [ ]:
ref_path = '../../process/integration/model/ref_model_5000/'
scanvae.save(ref_path, overwrite=True)